# Φ<sub>Flow</sub> Cookbook

This notebook lists useful code snippets.

## Import for NumPy, TensorFlow, PyTorch, Jax

In [ ]:
from phi.flow import *
from phi.tf.flow import *
from phi.torch.flow import *
from phi.jax.flow import *

## Select GPU or CPU

In [ ]:
gpus = backend.default_backend().list_devices('GPU')

assert backend.default_backend().set_default_device('CPU')


## Use 64 bit FP precision

In [ ]:
math.set_global_precision(64)
x64 = math.random_normal(batch(b=4))

with math.precision(32):  ## operations within this context will use 32 bit floats
    x32 = math.to_float(x64)


## Plot a `Tensor`

In [ ]:
data = math.random_uniform(spatial(x=8, y=6))
vis.plot(data);  ## or vis.show(data)


## Convert a `Tensor` to NumPy

In [ ]:
data.numpy(order='x,y')

## Compute Pair-wise Distances

In [ ]:
points = math.tensor([(0, 0), (0, 1), (1, 0)], instance('points'), channel('vector'))
distances = points - math.rename_dims(points, 'points', 'others')
math.print(distances)

## Construct a `CenteredGrid`

In [ ]:
zero_grid = CenteredGrid(0, 0, x=32, y=32, bounds=Box(x=1, y=1))
y_grid = CenteredGrid((0, 1), extrapolation.BOUNDARY, x=32, y=32)
noise_grid = CenteredGrid(Noise(), extrapolation.PERIODIC, x=32, y=32)
sin_curve = CenteredGrid(lambda x: math.sin(x), extrapolation.PERIODIC, x=100, bounds=Box(x=2 * PI))

vis.plot(zero_grid, y_grid, noise_grid, sin_curve);

## Construct a `StaggeredGrid`

In [ ]:
zero_grid = StaggeredGrid(0, 0, x=32, y=32, bounds=Box(x=1, y=1))
y_grid = StaggeredGrid((0, 1), extrapolation.BOUNDARY, x=32, y=32)
noise_grid = StaggeredGrid(Noise(), extrapolation.PERIODIC, x=32, y=32)
sin_curve = StaggeredGrid(lambda x: math.sin(x), extrapolation.PERIODIC, x=100, bounds=Box(x=2 * PI))

vis.plot(zero_grid, y_grid, noise_grid, sin_curve);


## Construct `StaggeredGrid` from NumPy Arrays

Given matching arrays `vx` and `vy`, we can construct a `StaggeredGrid`.
Note that the shapes of the arrays must [match the extrapolation](https://tum-pbs.github.io/PhiFlow/Staggered_Grids.html)!

In [ ]:
vx = math.tensor(np.zeros([33, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 33]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), extrapolation.BOUNDARY)

vx = math.tensor(np.zeros([32, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 32]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), extrapolation.PERIODIC)

vx = math.tensor(np.zeros([31, 32]), spatial('x,y'))
vy = math.tensor(np.zeros([32, 31]), spatial('x,y'))
StaggeredGrid(math.stack([vx, vy], channel('vector')), 0)


## BFGS Optimization

In [ ]:
def loss_function(x):
    return math.l2_loss(math.cos(x))

initial_guess = math.tensor([1, -1], math.batch('batch'))
math.minimize(loss_function, Solve('L-BFGS-B', 0, 1e-3, x0=initial_guess))
